In [1]:
import random
import os
import json
from IPython.display import display,JSON
import pickle

# 生成随机字符串
def generate_random_string(length: int = 10):
    letters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    return "".join(random.choice(letters) for i in range(length))


# 生成随机文件名
def generate_random_filename(extension):
    return generate_random_string(10) + "." + extension


# 生成随机整数
def generate_random_integer(start: int = -2147483648, end: int = 2147483647):
    return random.randint(start, end)


# 生成随机浮点数
def generate_random_float(start: float = -1e300, end: float = 1e300):
    return random.uniform(start, end)


# 生成随机布尔值
def generate_random_boolean():
    return random.choice([True, False])


# 生成随机列表
def generate_random_list(length: int = 10, item_generator=generate_random_integer):
    return [item_generator() for _ in range(length)]


# 生成随机字典
def generate_random_dict(
    length: int = 10,
    key_generator=generate_random_string,
    value_generator=generate_random_string,
):
    return {key_generator(): value_generator() for _ in range(length)}


# dump to string
def dump_to_string(data):
    # print(data, type(data))
    if data is None:
        # print("data is None")
        return "{}"
    if isinstance(data, str):
        # print("data is str")
        return '"' + data + '"'
    if isinstance(data, bool):
        # print("data is bool")
        return str(data).lower()
    if isinstance(data, (int, float)):
        # print("data is number")
        return str(data)
    if isinstance(data, (list, tuple)):
        # print("data is list")
        return "{" + ",".join(dump_to_string(item) for item in data) + "}"
    if isinstance(data, dict):
        # print("data is map")
        return (
            "{"
            + ",".join(
                "{" + "{}, {}".format(dump_to_string(k), dump_to_string(v)) + "}"
                for k, v in data.items()
            )
            + "}"
        )
    return "{}"

class CppStructure:
    def __init__(self, struct_name : str, struct_field_names : list, struct_field_types : list, type_generate_map : dict, **args):
        if "base_class" in args:
            self.base_class = args["base_class"]
        else:
            self.base_class = ""
        self.struct_name = struct_name
        self.struct_field_names = struct_field_names
        self.struct_field_types = struct_field_types
        self.type_generate_map = type_generate_map
        assert(len(struct_field_names) == len(struct_field_types))
    
    def replace_whit(self, x: str):
        x = x.replace("$struct_name", self.struct_name)
        x = x.replace("$field_num", str(len(self.struct_field_names)))
        x = x.replace("$field_names", ",".join(self.struct_field_names))
        x = x.replace("$field_types", ",".join(self.struct_field_types))
        x = x.replace("$base_class", self.base_class)
        for i in range(len(self.struct_field_names)):
            x = x.replace("$field_" + str(i), self.struct_field_names[i])
            x = x.replace("$field_type_" + str(i), self.struct_field_types[i])
        return x

    def generate_cpp_structure_text(self, base_class: str = "", head_fill: str = "", tail_fill: str = "", **args):
        ret = ""
        if self.base_class != "":
            ret = self.base_class + " " + self.struct_name + " : " + self.base_class + "{\n"
        else:
            ret = "struct " + self.struct_name + " {\n"

        # 填充结构体头
        if head_fill != "":
            head_fill = self.replace_whit(head_fill)
            ret += head_fill + "\n"

        # 生成字段定义
        for i in range(len(self.struct_field_names)):
            ret += "    " + self.struct_field_types[i] + " " + self.struct_field_names[i]
            if "no_initialize" in args and args["no_initialize"]:
                ret += " = {};\n"
            else:
                value = self.type_generate_map[self.struct_field_types[i]]()
                ret += " = " + dump_to_string(value) + ";\n"

        # 填充结构体尾
        if tail_fill != "":
            tail_fill = self.replace_whit(tail_fill)
            ret += tail_fill + "\n"
        ret += "};\n"
        return ret
    
    def generate_random_value(self):
        json_data = dict()
        for i in range(len(self.struct_field_names)):
            json_data[self.struct_field_names[i]] = self.type_generate_map[self.struct_field_types[i]]()
        return json_data
    
    def generate_json(self):
        json_data = self.generate_random_value()
        return json.dumps(json_data, indent=4)

# 生成随机结构体
def generate_random_structure(
    field_num: int,
    field_avaliable_types: dict,
    struct_name: str = "",
    base_class: str = "",
    head_fill: str = "",
    tail_fill: str = "",
    **args
):
    json_data = dict()
    # 生成结构体名
    if struct_name == "":
        struct_name = "TestStruct" + generate_random_string(10)

    # 生成字段名
    field_names = []
    if args.get("field_names") is not None:
        field_names = args["field_names"]
        if len(field_names) > field_num:
            field_names = field_names[:field_num]
        elif len(field_names) < field_num:
            field_names += ["f" + str(i) for i in range(len(field_names), field_num)]
    else:
        field_names = ["f" + str(i) for i in range(field_num)]
    # 生成字段类型
    field_types = []
    if args.get("field_types") is not None:
        field_types = args["field_types"]
        if len(field_types) > field_num:
            field_types = field_types[:field_num]
        elif len(field_types) < field_num:
            field_types += [
                random.choice(list(field_avaliable_types.keys()))
                for _ in range(len(field_types), field_num)
            ]
    else:
        field_types = [
            random.choice(list(field_avaliable_types.keys())) for _ in range(field_num)
        ]
    
    structd = CppStructure(struct_name=struct_name, struct_field_names=field_names, struct_field_types=field_types, type_generate_map=field_avaliable_types, base_class=base_class)

    return structd


def generate_random_enum(enum_count: int, enum_name: str = "", enum_values: list = []):
    if enum_name == "":
        enum_name = "TestEnum" + generate_random_string(5)
    if enum_count > len(enum_values):
        enum_values += [enum_name + str(i) for i in range(len(enum_values), enum_count)]
    return (
        "enum class " + enum_name + " {\n    " + ",\n    ".join(enum_values) + "\n};\n"
    )

In [11]:

# 常用类型
field_avaliable_types = {
    "int": generate_random_integer,
    "double": generate_random_float,
    "bool": generate_random_boolean,
    "std::string": generate_random_string,
    "std::vector<int>": lambda len=10: generate_random_list(len, generate_random_integer),
    "std::map<std::string, int>": lambda len=10: generate_random_dict(
        len, generate_random_string, generate_random_integer
    ),
}


# print(genrate_random_enum(62))
# 生成随机struct
cppStructures = []
for i in range(1, 5):
    ret : CppStructure = generate_random_structure(
        generate_random_integer(21, 81),
        field_avaliable_types,
        tail_fill=(
            "NEKO_SERIALIZER($field_names)\nNEKO_DECLARE_PROTOCOL($struct_name, JsonSerializer)"
        ),
        struct_name="TestStruct" + str(i))
    field_avaliable_types["TestStruct" + str(i)] = ret.generate_random_value
    cppStructures.append(ret)

In [12]:
print(cppStructures)
print(field_avaliable_types)

[<__main__.CppStructure object at 0x743f215d5690>, <__main__.CppStructure object at 0x743f202b4820>, <__main__.CppStructure object at 0x743f20299390>, <__main__.CppStructure object at 0x743f2029b040>]
{'int': <function generate_random_integer at 0x743f215224d0>, 'double': <function generate_random_float at 0x743f21521f30>, 'bool': <function generate_random_boolean at 0x743f21521ea0>, 'std::string': <function generate_random_string at 0x743f21521bd0>, 'std::vector<int>': <function <lambda> at 0x743f0bf55bd0>, 'std::map<std::string, int>': <function <lambda> at 0x743f0bf54550>, 'TestStruct1': <bound method CppStructure.generate_random_value of <__main__.CppStructure object at 0x743f215d5690>>, 'TestStruct2': <bound method CppStructure.generate_random_value of <__main__.CppStructure object at 0x743f202b4820>>, 'TestStruct3': <bound method CppStructure.generate_random_value of <__main__.CppStructure object at 0x743f20299390>>, 'TestStruct4': <bound method CppStructure.generate_random_value

In [13]:
for i in cppStructures:
    print(i.generate_cpp_structure_text( tail_fill=(
            "NEKO_SERIALIZER($field_names)\nNEKO_DECLARE_PROTOCOL($struct_name, JsonSerializer)"
        ),no_initialize=True))

struct TestStruct1 {
    std::map<std::string, int> f0 = {};
    std::string f1 = {};
    int f2 = {};
    double f3 = {};
    std::map<std::string, int> f4 = {};
    int f5 = {};
    double f6 = {};
    std::map<std::string, int> f7 = {};
    double f8 = {};
    std::string f9 = {};
    int f10 = {};
    int f11 = {};
    std::vector<int> f12 = {};
    std::map<std::string, int> f13 = {};
    double f14 = {};
    std::map<std::string, int> f15 = {};
    int f16 = {};
    bool f17 = {};
    bool f18 = {};
    double f19 = {};
    std::string f20 = {};
    int f21 = {};
    int f22 = {};
    std::string f23 = {};
    bool f24 = {};
    std::map<std::string, int> f25 = {};
    std::vector<int> f26 = {};
    int f27 = {};
    std::string f28 = {};
    bool f29 = {};
    std::string f30 = {};
    double f31 = {};
    std::map<std::string, int> f32 = {};
    std::vector<int> f33 = {};
    std::string f34 = {};
    std::string f35 = {};
    double f36 = {};
    double f37 = {};
    double f3

In [17]:
for i in range(1, 10):
    with open(".ipynb_checkpoints/test_data_" + str(i) + ".json", "w") as file:
        file.write(cppStructures[-1].generate_json())